# Lane Detection using Hough Transform in OpenCV

In [ ]:
from pathlib import Path
import cv2
import numpy as np
from tqdm.notebook import tqdm

print(f"OpenCV version: {cv2.__version__}")

In [ ]:
input_path = Path("data/lane1-straight.mp4")
output_path = Path(f"output/{input_path.stem}-out{input_path.suffix}")

from moviepy.editor import *

clip = VideoFileClip(filename=str(input_path))
clip.ipython_display(width=800)

In [25]:
from typing import List, Tuple, Optional


def process_frame(frame: np.ndarray) -> np.ndarray:
    """
    Function for processing and annotating a single frame.

    Args:
        frame:
            Frame to process.

    Returns:
        Processed frame.
    """

    # Convert colored frame to grayscale
    img_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # Select specific intensity range (Thresholding)
    img_gray_select = cv2.inRange(img_gray, 150, 255)

    # Mask region of interest (ROI)
    roi_vertices = np.array([[[100, 540], [900, 540], [525, 330], [440, 330]]])
    img_gray_roi = mask_roi(frame=img_gray_select, vertices=roi_vertices)

    # Edge detection
    img_canny = cv2.Canny(image=img_gray_roi, threshold1=50, threshold2=100)
    img_canny = cv2.GaussianBlur(src=img_canny, ksize=(5, 5), sigmaX=0)

    # Get Hough lines.
    hough_lines = cv2.HoughLinesP(
        image=img_canny,
        rho=1,
        theta=(np.pi / 180),
        threshold=100,
        lines=np.array([]),
        minLineLength=50,
        maxLineGap=300,
    )

    # Extrapolate Hough lines to left and right lanes.
    img_lanes = get_lanes(frame, hough_lines, roi_upper_border=330, roi_lower_border=540)

    # Combine original input frame with the extrapolated lanes.
    image_result = cv2.addWeighted(src1=frame, alpha=1, src2=img_lanes, beta=0.4, gamma=0.0)

    return image_result


def mask_roi(frame: np.ndarray, vertices: np.ndarray) -> np.ndarray:
    """
    Mask the region of interest (ROI) from a defined list of vertices.

    Args:
        frame:
            Input grayscale image.
        vertices:
            List containing the vertices of the mask polygon.

    Returns:
        Binary image masked with ROI.
    """

    mask = np.zeros_like(frame)
    cv2.fillPoly(img=mask, pts=vertices, color=255)

    return cv2.bitwise_and(src1=frame, src2=mask)


def get_lanes(frame: np.ndarray, lines: np.ndarray, roi_upper_border: int, roi_lower_border: int) -> np.ndarray:
    """
    Function for getting the final extrapolated left and right lanes from the Hough lines.

    Args:
        frame:
            Frame to process.
        lines:
            List containing line coordinates calculated with Hough Transform.
        roi_upper_border:
            Upper Y value of ROI.
        roi_lower_border:
            Lower Y value of ROI.

    Returns:
        Image containing only the extrapolated left and right lines.
    """

    # Extract left and right lanes.
    lines_left, lines_right = separate_left_right_lines(lines)
    lane_left = extrapolate_lines(lines=lines_left, upper_border=roi_upper_border, lower_border=roi_lower_border)
    lane_right = extrapolate_lines(lines=lines_right, upper_border=roi_upper_border, lower_border=roi_lower_border)

    img_lanes = np.zeros(shape=(frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    if lane_left is not None and lane_right is not None:
        draw_lane_roi(frame=img_lanes, lanes=[lane_left, lane_right])

    return img_lanes


def separate_left_right_lines(lines) -> Tuple[List[List[int]]]:
    """
    Function separating left and right lines depending on the slope.

    Args:
        lines:
            List containing line coordinates calculated with Hough Transform.

    Returns:
        Left and right lines in separate lists.
    """

    left_lines = []
    right_lines = []

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if y1 > y2:  # Left line (negative slope)
                    left_lines.append([x1, y1, x2, y2])
                elif y1 < y2:  # Right line (positive slope)
                    right_lines.append([x1, y1, x2, y2])

    return left_lines, right_lines


def extrapolate_lines(lines: List[int], upper_border: int, lower_border: int) -> Optional[List[int]]:
    """
    Function extrapolating lines keeping in mind the lower and upper border of ROI.

    Args:
        lines:
            List containing line coordinates calculated with Hough Transform.
        upper_border:
            Upper Y value of ROI.
        lower_border:
            Lower Y value of ROI.

    Returns:
        Optional: List containing starting and ending coordinates of the extrapolated lane.
    """

    slopes = []
    consts = []

    if len(lines):
        for x1, y1, x2, y2 in lines:
            slopes.append((y1 - y2) / (x1 - x2))
            consts.append(y1 - slopes[-1] * x1)

        avg_slope = sum(slopes) / max(1, len(slopes))
        avg_consts = sum(consts) / max(1, len(consts))

        # Calculate average intersection at lower_border.
        x_lower_point = int((lower_border - avg_consts) / avg_slope)

        # Calculate average intersection at upper_border.
        x_upper_point = int((upper_border - avg_consts) / avg_slope)

        return [x_lower_point, lower_border, x_upper_point, upper_border]

    return None


def draw_lane_roi(frame: np.ndarray, lanes: List[List[int]]) -> None:
    """
    Function filling in the lane ROI area.

    Args:
        frame:
            Frame on which to draw the lanes.
        lanes:
            List containing the left and right extrapolated lanes.

    Returns:
        -
    """

    points = []

    x1, y1, x2, y2 = lanes[0]
    points.append([x1, y1])
    points.append([x2, y2])

    x1, y1, x2, y2 = lanes[1]
    points.append([x2, y2])
    points.append([x1, y1])

    points = np.array(points, dtype="int32")

    cv2.fillPoly(img=frame, pts=[points], color=(0, 255, 0))

In [26]:
# Initialize video capture
video_cap = cv2.VideoCapture(str(input_path))
if not video_cap.isOpened():
    print("Error opening video stream or file.")
else:
    # Get video properties
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))

In [ ]:
# progress_bar = tqdm(total=frame_count, desc="Processing")
while True:
    has_frame, frame = video_cap.read()
    if not has_frame:
        break

    frame_processed = process_frame(frame=frame)
    video_out.write(frame_processed)

    # progress_bar.update(1)

video_out.release()